# Approach 1

# LDA

In [29]:
import pandas as pd
df = pd.read_json('realDonaldTrump.jsonl', lines=True)
df = df['full_text']
df.head()

0    As everybody is aware, the past Administration...
1    “This isn’t some game. You are screwing with t...
2    I have been briefed on the U.S. C-130 “Hercule...
3    Congratulations @SecPompeo! https://t.co/ECrMG...
4    A Rigged System - They don’t want to turn over...
Name: full_text, dtype: object

In [30]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

n_components =10
n_top_words = 10


For  both tokenization and occurrence counting we use the CountVectorizer class, we use the default settings due to they behaivier with small corpora. max_df is set to 1 meaning non word will be ignored.

In [35]:
tf_vectorizer = CountVectorizer(stop_words='english',token_pattern = r'\b[a-zA-Z]{3,}\b',lowercase = True)
tf = tf_vectorizer.fit_transform(df)


For a large corpora we will usually use 'online' learning methdod but we instead use batch variational Bayes method since we use all training data in each Expectation Maximisation update. Random_State is enabled for debuggin pruposes. And the default values are adecuate for our application.

In [36]:
lda = LatentDirichletAllocation(learning_method='batch', random_state=0)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [23]:
# Just to print top words of our model.
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [37]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0: people great just news fake amp country good korea north
Topic #1: great people amp good https state like border today tough
Topic #2: https great russia people trump whitehouse story border happy big
Topic #3: justice house great committee trump collusion fbi russia campaign law
Topic #4: korea north trade time nuclear ago war comey billion work
Topic #5: great https house president today news fake honor military america
Topic #6: great amp people country time years north meeting korea military
Topic #7: https great president amp going trump billion security job mccabe
Topic #8: https jobs great american trade workers country america today law
Topic #9: https border trump whitehouse maga thank wall new congratulations higher



 If we wanted to use TFIDF we could do it with the code below.The raw documents are converted into DTM, possibly as raw counts or TF-IDF form. TfidfVectorizer normalizes its results, i.e. each vector in its output has norm 1. This is done so that dot-products on the rows are cosine similarities. For simplicity we wont implement TFIDF in this approach.



In [ ]:
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(df)
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)

In [38]:
from __future__ import print_function
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
0      17.838796        1       1 -0.087558 -0.027342
5      11.916396        1       2  0.099857 -0.057349
6      10.971991        1       3 -0.063860  0.075493
2       9.250225        1       4  0.031885 -0.033364
7       9.098384        1       5 -0.014161  0.019616
9       8.992743        1       6  0.070686  0.041158
3       8.721444        1       7 -0.048925 -0.153584
8       8.445465        1       8  0.066041  0.033733
1       7.601444        1       9  0.060673  0.048461
4       7.163111        1      10 -0.114637  0.053177, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
962   Default  82.000000        https  82.000000  30.0000  30.0000
1371  Default  20.000000        north  20.000000  29.0000  29.0000
1110  Default  22.000000        korea  22.000000  28.0000  28.0000
1070  Default  16.000000         jobs  16.000000  27.0000  27.0000
957   Default  21.000000        house  21.000000  26.0000  26.0000
80    Default  14.000000     american  14.000000  25.0000  25.0000
2067  Default  26.000000        trade  26.000000  24.0000  24.0000
1086  Default  10.000000      justice  10.000000  23.0000  23.0000
83    Default  37.000000          amp  37.000000  22.0000  22.0000
2097  Default  24.000000        trump  24.000000  21.0000  21.0000
192   Default  12.000000      billion  12.000000  20.0000  20.0000
370   Default   5.000000    committee   5.000000  19.0000  19.0000
856   Default  80.000000        great  80.000000  18.0000  18.0000
1300  Default   9.000000     minister   9.000000  17.0000  17.0000
1379  Default   7.000000      nuclear   7.000000  16.0000  16.0000
734   Default   9.000000          fbi   9.000000  15.0000  15.0000
1225  Default  13.000000         maga  13.000000  14.0000  14.0000
1741  Default  18.000000       russia  18.000000  13.0000  13.0000
1551  Default  10.000000        prime  10.000000  12.0000  12.0000
79    Default  15.000000      america  15.000000  11.0000  11.0000
2048  Default  19.000000         time  19.000000  10.0000  10.0000
2238  Default   6.000000      workers   6.000000   9.0000   9.0000
1357  Default  25.000000         news  25.000000   8.0000   8.0000
355   Default  16.000000    collusion  16.000000   7.0000   7.0000
2257  Default  16.000000        years  16.000000   6.0000   6.0000
2214  Default  16.000000   whitehouse  16.000000   5.0000   5.0000
2188  Default   5.000000          war   5.000000   4.0000   4.0000
1130  Default  13.000000          law  13.000000   3.0000   3.0000
51    Default   5.000000          ago   5.000000   2.0000   2.0000
841   Default  19.000000        going  19.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
1371  Topic10   7.217482        north  20.323341   1.6010  -4.1839
1110  Topic10   7.866354        korea  22.522625   1.5843  -4.0978
1194  Topic10   2.831518         long   7.233678   1.6983  -5.1196
525   Topic10   2.523922      deficit   6.390301   1.7073  -5.2346
570   Topic10   2.831515          did   8.238843   1.5682  -5.1196
1897  Topic10   1.935135        spoke   5.035219   1.6799  -5.5002
1990  Topic10   2.140905      tariffs   5.818598   1.6364  -5.3992
2067  Topic10   6.215928        trade  26.373729   1.1910  -4.3333
2048  Topic10   4.212765         time  19.937590   1.0817  -4.7223
192   Topic10   2.909733      billion  12.341559   1.1913  -5.0924
152   Topic10   2.140898        badly   6.644263   1.5037  -5.3992
2235  Topic10   2.831527         work  13.356180   1.0851  -5.1196
360   Topic10   3.216582        comey  21.131606   0.7538  -4.9921
316   Topic10   2.632719        china  14.975909   0.8978  -5.1924
1279  Topic10   2.140891        memos   7.916595   1.3285  -5.3992
542   Topic10   2.696074    democrats  18.500653   0.7102  -5.1686
719   Topic10   2.140890  

One way to validate our model is by Human-in-the-Loop method. This means that a human tries to identify documents with potential for systems. If you place close attention to Topic #10 you can easily spot that the general topic is about the north korea war which makes sense because that's what we are seeing in the news.